---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

# Plotting with matplotlib kills the grader, so here is a flag to turn it off for submissions
plot_flag = False
if plot_flag: 
    import matplotlib.pyplot as plt
    import seaborn as sns

In [2]:
def degree_dist(G, plot_flag = 1, title_text = 'Degree Distribution'):
    degrees = dict(G.degree())
    degree_values = sorted(set(degrees.values()))
    
    histogram = [list(dict(degrees).values()).count(i)/
             float(nx.number_of_nodes(G))
             for i in degree_values]
    
    if plot_flag:
        
        plt.bar(degree_values, histogram)
        plt.xlabel('Degree')
        plt.ylabel('Fraction of nodes')
        plt.title(title_text)
        plt.show()
    
    return degree_values, histogram

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

## Calculate a few things

Explore these graphs. What is going to be useful? 

* Average path length    
* Average clustering coefficient
* Degree distribution

In [4]:
avg_paths = [nx.average_shortest_path_length(G) for G in P1_Graphs]
avg_paths

[4.099161161161161,
 5.089871871871872,
 9.378702269692925,
 3.1048046283934134,
 5.0785509568313305]

In [5]:
avg_clust = [nx.average_clustering(G) for G in P1_Graphs]
avg_clust

[0.03167539146454044,
 0.5642419635919628,
 0.4018222222222227,
 0.03780379975223251,
 0.0033037037037037037]

In [6]:
avg_df = pd.DataFrame(data={'avg_paths':avg_paths,
                            'avg_clust':avg_clust,
                            'index':np.arange(5)})
avg_df

,avg_clust,avg_paths,index
0,0.031675,4.099161,0
1,0.564242,5.089872,1
2,0.401822,9.378702,2
3,0.037804,3.104805,3
4,0.003304,5.078551,4


In [7]:
if plot_flag:
    plt.plot(avg_paths, avg_clust, 'o')
    for idx, row in avg_df.iterrows():
        plt.text(row["avg_paths"]+.1, row["avg_clust"], '{}'.format(row["index"]))
    plt.xlabel('Average path length');
    plt.ylabel('Average clustering coefficient')
    plt.show()

In [8]:
index_counter = 0
histogram_list = []
degree_values_list = []
for G in P1_Graphs:
    degree_values, histogram = degree_dist(G, 
                                           plot_flag = plot_flag,
                                           title_text = 'Degree Distribution for {}'.format(index_counter))
    index_counter += 1
    
    degree_values_list.append(degree_values)
    histogram_list.append(histogram)

`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:

* Preferential Attachment (`'PA'`)  
    * small shortest path  
    * very small clustering coefficient  
    * long histogram with a handful of very high degree nodes
    * 0 and 3
* Small World with low probability of rewiring (`'SW_L'`)    
    * Very short paths  
    * High clustering coefficient  
    * 1 and 2
* Small World with high probability of rewiring (`'SW_H'`)   
    * Smaller shortest path (rapidly)  
    * Smaller average clustering (slowly)  
    * 4

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
def graph_identification():
    
    algorithm_list = []
    
    for i in range(5):
        if len(histogram_list[i]) > 12:
            print(i, 'Histogram')
            algorithm_list.append('PA')
        elif avg_clust[i] < 0.05:
            print(i, 'Clust low')
            algorithm_list.append('SW_H')
        else:
            print(i, 'Clust high')
            algorithm_list.append('SW_L')
    
    
    return algorithm_list

In [10]:
graph_identification()

0 Histogram
1 Clust high
2 Clust high
3 Histogram
4 Clust low


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [11]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

* 1a. Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and  
* 1b. Predict whether or not these individuals are receiving a management position salary.  
    * To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.
    * Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.
    * The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

My notes:

Since the final answer is going to be a Series, I'm going to take the hint that I should transform the network data into a pandas DataFrame, where each row is an individual (a node in the graph). Start with initializing a DataFrame and filling it with the attributes assigned to each node. First look at the nodes and see what attributes are assigned to them. 

In [12]:
G.nodes(data=True)[:5]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0})]

In [13]:
G_df = pd.DataFrame(data={'Department':[node[1]['Department'] for node in G.nodes(data=True)],
                          'ManagementSalary':[node[1]['ManagementSalary'] for node in G.nodes(data=True)]},
                    index=G.nodes())
G_df.head(10)

,Department,ManagementSalary
0,1,0.0
1,1,NaN
2,21,NaN
3,21,1.0
4,21,1.0
5,25,NaN
6,25,1.0
7,14,0.0
8,14,NaN
9,14,0.0


In [14]:
# Mask for null entries in ManagementSalary
null_mask = G_df['ManagementSalary'].isnull()
G_df[null_mask].head(10)

,Department,ManagementSalary
1,1,NaN
2,21,NaN
5,25,NaN
8,14,NaN
14,4,NaN
18,1,NaN
27,11,NaN
30,11,NaN
31,11,NaN
34,11,NaN


In [15]:
# Take a look at the number of employees in each department, split by salary info.
if plot_flag:
    plt.figure(figsize=(15,7))
    sns.countplot(x = 'Department', 
                  hue = 'ManagementSalary',
                  data = G_df.fillna(value = 'None'))
    plt.show()    

I need to generate more features. What else can I use? Look back at content from week 3.

    * Degrees
    * Degree centrality  
    * Closeness centrality  
    * Betweenness centrality  
    * Clustering coefficient

Start with these. 

In [16]:
G_df['Degree']         = pd.Series(G.degree())
G_df['Degree_Cent']    = pd.Series(nx.degree_centrality(G))
G_df['Closeness_Cent'] = pd.Series(nx.closeness_centrality(G))
G_df['Between_Cent']   = pd.Series(nx.betweenness_centrality(G))
G_df['Cluster_Coeff']  = pd.Series(nx.clustering(G))
G_df.head(10)

,Department,ManagementSalary,Degree,Degree_Cent,Closeness_Cent,Between_Cent,Cluster_Coeff
0,1,0.0,44,0.043825,0.421991,0.001124,0.276423
1,1,NaN,52,0.051793,0.422360,0.001195,0.265306
2,21,NaN,95,0.094622,0.461490,0.006570,0.297803
3,21,1.0,71,0.070717,0.441663,0.001654,0.384910
4,21,1.0,96,0.095618,0.462152,0.005547,0.318691
5,25,NaN,171,0.170319,0.501484,0.030995,0.107002
6,25,1.0,115,0.114542,0.475805,0.012387,0.155183
7,14,0.0,72,0.071713,0.420156,0.002818,0.287785
8,14,NaN,37,0.036853,0.413151,0.000557,0.447059
9,14,0.0,40,0.039841,0.356196,0.000280,0.425320


Now I need to predict whether the employees without salary data have a management salary. 

Try a few machine learning methods. Let's set up a train-test split of the data with salary info, though I will end up training on all of it. 

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
features = ['Department','Degree','Degree_Cent','Closeness_Cent','Between_Cent','Cluster_Coeff']

# These will be our final training arrays
X_sal        = G_df[~null_mask][features].values
y_sal        = G_df[~null_mask]['ManagementSalary'].values.ravel()

scaler = MinMaxScaler()
scaler.fit(X_sal)
X_sal_scaled = scaler.transform(X_sal)


# We will make predictions on this array
X_null        = G_df[null_mask][features].values
X_null_scaled = scaler.transform(X_null)

In [19]:
# Generate smaller train and test arrays for picking a model
X_train, X_test, y_train, y_test = train_test_split(X_sal,y_sal)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

Linear regression. Unfortunately this doesn't really make sense for this data because the Department number is not necessarily ordinal. 

In [20]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train_scaled,y_train)
print('Training score:', linreg.score(X_train_scaled,y_train))
print('Testing score:',  linreg.score(X_test_scaled ,y_test ))

Training score: 0.287093288628
Testing score: 0.37129945111


Let's try SVC linear kernel instead. This shows some improvement. Try a few more. 

In [21]:
from sklearn.svm import SVC
svm = SVC(kernel='linear',C=1).fit(X_train_scaled,y_train)
print('Training score:', svm.score(X_train_scaled,y_train))
print('Testing score:', svm.score(X_test_scaled  ,y_test ))

Training score: 0.882978723404
Testing score: 0.867724867725


In [22]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf',C=1).fit(X_train_scaled,y_train)
print('Training score:', svm.score(X_train_scaled,y_train))
print('Testing score:', svm.score(X_test_scaled,y_test))

Training score: 0.858156028369
Testing score: 0.835978835979


Try random forest classifier.

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train_scaled,y_train)
print('Training score:', rf.score(X_train_scaled,y_train))
print('Testing score:', rf.score(  X_test_scaled,y_test ))

Training score: 0.989361702128
Testing score: 0.936507936508


The SVC models and the random forest are scoring about the same. I'll go with the linear SVC. Now I am going to use all of the entries with salary data as `X_train`.

In [24]:
svm = SVC(kernel='rbf',C=1,probability=True).fit(X_sal_scaled,y_sal)
print('Training score:', svm.score(X_sal_scaled,y_sal))

Training score: 0.864541832669


In [25]:
final_proba = pd.Series(data = svm.predict_proba(X_null_scaled)[:,1],
                        index = G_df[null_mask].index.values)
final_proba.head(10)

1     0.096478
2     0.724637
5     0.999990
8     0.106418
14    0.305011
18    0.149189
27    0.259458
30    0.315322
31    0.148597
34    0.115289
dtype: float64

In [26]:
def salary_predictions():
    
    # Your Code Here
    
    return final_proba

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [27]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

In [28]:
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


In [29]:
future_connections['Future Connection'].unique()

array([  0.,   1.,  nan])

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

The first thing I need to do is split up the nodes for each edge into two columns. 

In [30]:
future_connections['Node1'] = [idx[0] for idx in future_connections.index.values]
future_connections['Node2'] = [idx[1] for idx in future_connections.index.values]

future_connections.head()

,Future Connection,Node1,Node2
"(6, 840)",0.0,6,840
"(4, 197)",0.0,4,197
"(620, 979)",0.0,620,979
"(519, 872)",0.0,519,872
"(382, 423)",0.0,382,423


Calculate features:

* Number of common neighbors of $X$ and $Y$  
* Jaccard coefficient  
* Resource allocation of nodes $X$ and $Y$  
* Adamic-adar index  
* Preferential attachment  

Leave out the community resources for now. 

In [31]:
# Number of common neighbors
future_connections['num_common_neighbors'] = [len(sorted(nx.common_neighbors(G,idx[0],idx[1]))) 
                                              for idx in future_connections.index.values]

In [32]:
# Jaccard coefficient
preds = nx.jaccard_coefficient(G,future_connections.index.values)
preds = list(preds)
future_connections['jaccard'] = [pred[2] for pred in preds]

In [33]:
# Resource allocation
preds = nx.resource_allocation_index(G,future_connections.index.values)
preds = list(preds)
future_connections['resource_allocation'] = [pred[2] for pred in preds]

In [34]:
# Adamic-adar index
preds = nx.resource_allocation_index(G,future_connections.index.values)
preds = list(preds)
future_connections['adamic_adar'] = [pred[2] for pred in preds]

In [35]:
# Preferential attachment
preds = nx.preferential_attachment(G,future_connections.index.values)
preds = list(preds)
future_connections['pref_attach'] = [pred[2] for pred in preds]

In [36]:
future_connections.head()

,Future Connection,Node1,Node2,num_common_neighbors,jaccard,resource_allocation,adamic_adar,pref_attach
"(6, 840)",0.0,6,840,9,0.073770,0.136721,0.136721,2070
"(4, 197)",0.0,4,197,2,0.015504,0.008437,0.008437,3552
"(620, 979)",0.0,620,979,0,0.000000,0.000000,0.000000,28
"(519, 872)",0.0,519,872,2,0.060606,0.039726,0.039726,299
"(382, 423)",0.0,382,423,0,0.000000,0.000000,0.000000,205


Identify the null entries in Future Connection. 

In [37]:
null_mask = future_connections['Future Connection'].isnull()
future_connections[null_mask].head(10)

,Future Connection,Node1,Node2,num_common_neighbors,jaccard,resource_allocation,adamic_adar,pref_attach
"(107, 348)",NaN,107,348,2,0.009009,0.025562,0.025562,884
"(542, 751)",NaN,542,751,0,0.000000,0.000000,0.000000,126
"(20, 426)",NaN,20,426,10,0.081967,0.082016,0.082016,4440
"(50, 989)",NaN,50,989,0,0.000000,0.000000,0.000000,68
"(942, 986)",NaN,942,986,0,0.000000,0.000000,0.000000,6
"(324, 857)",NaN,324,857,0,0.000000,0.000000,0.000000,76
"(13, 710)",NaN,13,710,6,0.031250,0.111676,0.111676,3600
"(19, 271)",NaN,19,271,6,0.044776,0.050306,0.050306,5040
"(319, 878)",NaN,319,878,0,0.000000,0.000000,0.000000,48
"(659, 707)",NaN,659,707,0,0.000000,0.000000,0.000000,120


In [38]:
features = ['Node1','Node2','jaccard','resource_allocation','num_common_neighbors','adamic_adar']

# These will be our final training arrays
X        = future_connections[~null_mask][features].values
y        = future_connections[~null_mask]['Future Connection'].values.ravel()

scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)


# We will make predictions on this array
X_null        = future_connections[null_mask][features].values
X_null_scaled = scaler.transform(X_null)

In [39]:
# Generate smaller train and test arrays for picking a model
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

Linear regression. Not very good.

In [40]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train_scaled,y_train)
print('Training score:', linreg.score(X_train_scaled,y_train))
print('Testing score:',  linreg.score(X_test_scaled ,y_test ))

Training score: 0.447560024965
Testing score: 0.44790154254


Let's try SVC linear kernel instead.
These are taking forever so I'm going to skip these.

In [41]:
# from sklearn.svm import SVC
# svm = SVC(kernel='linear',C=1).fit(X_train_scaled,y_train)
# print('Training score:', svm.score(X_train_scaled,y_train))
# print('Testing score:', svm.score(X_test_scaled  ,y_test ))

In [42]:
# from sklearn.svm import SVC
# svm = SVC(kernel='rbf',C=1).fit(X_train_scaled,y_train)
# print('Training score:', svm.score(X_train_scaled,y_train))
# print('Testing score:', svm.score(X_test_scaled,y_test))

Try random forest classifier. Wow, this has a really high score! Go with random forest.

In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train_scaled,y_train)
print('Training score:', rf.score(X_train_scaled,y_train))
print('Testing score:', rf.score(  X_test_scaled,y_test ))

Training score: 0.993954504095
Testing score: 0.955177760307


In [47]:
rf = RandomForestClassifier().fit(X_scaled,y)
print('Training score:', rf.score(X_scaled,y))

Training score: 0.993686089743


In [49]:
final_proba_2 = pd.Series(data = rf.predict_proba(X_null_scaled)[:,1],
                        index = future_connections[null_mask].index.values)
final_proba_2.head(10)

(107, 348)    0.0
(542, 751)    0.0
(20, 426)     0.6
(50, 989)     0.0
(942, 986)    0.0
(324, 857)    0.0
(13, 710)     0.2
(19, 271)     0.1
(319, 878)    0.0
(659, 707)    0.0
dtype: float64

In [50]:
def new_connections_predictions():
    
    # Your Code Here
    
    return final_proba_2